# Case X.
Lauri Marjanen, Team 10<br>
Neural Networks for Machine Learning Applications<br>
[Helsinki Metropolia University of Applied Sciences](http://www.metropolia.fi/en/)<br>

In [102]:
import copy
%pylab inline
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("tensorflow", tf.__version__)

Populating the interactive namespace from numpy and matplotlib
tensorflow 2.7.0


C:\ProgramData\Anaconda3\envs\Anaconda ML\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['hist', 'copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# Background

The aim of this Notebook is ...

# Data

Different data processing methods attempted:
set image color = grayscale to change image dim from 128*128*3 to 128*128*1, did not help accuracy

In [103]:
batch_size = 128
img_height = 128
img_width = 128

curDir = os.getcwd()
train_dir = curDir + '/input/train'
test_dir = curDir + '/input/test'
val_dir = curDir + '/input/val'

# Training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split = 0.2,
  subset = "training",
  seed = 123,
  image_size = (img_height, img_width),
  batch_size = batch_size)


# Validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 123,
  image_size = (img_height, img_width),
  batch_size = batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.
Found 624 files belonging to 2 classes.


In [104]:
datagen = ImageDataGenerator( # Usually xray machines force you to stand still in a specific position,
                              # so it does not make sense for the data to be augmented too much
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.11,
  zoom_range=0.11,
  fill_mode='nearest',
  validation_split=0.2
)
train_gen_ds = datagen.flow_from_directory(train_dir, class_mode='binary',
                                         subset = "training",
                                         target_size = (img_height, img_width),
                                         batch_size = batch_size)
val_gen_ds = datagen.flow_from_directory(train_dir, class_mode='binary',
                                         subset = "validation",
                                         target_size = (img_height, img_width),
                                         batch_size = batch_size)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


In [105]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Model and training

The following model was used: "vanilla" 2d image model, 2d image model with generated data,
pretrained model with a normal data , and pretrained model with generated image data

In [106]:
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16,3, padding='same', activation= 'relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32,3, padding='same', activation= 'relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64,3, padding='same', activation= 'relu'),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.RMSprop(learning_rate= 1e-4),
              metrics= ['accuracy'])

gen_model = keras.models.clone_model(model)
gen_model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.RMSprop(learning_rate= 1e-4),
              metrics= ['accuracy'])

model.summary()
gen_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_394 (Conv2D)         (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_395 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_396 (Conv2D)         (None, 32, 32, 64)       

In [107]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

base_model = tf.keras.applications.InceptionV3(input_shape=(img_width,img_height,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)
#Modeling the child

inputs = tf.keras.Input(shape = (img_width, img_height,3))
holder = preprocess_input(inputs)
holder = base_model(holder, training = False)
holder = global_average_layer(holder)
holder = tf.keras.layers.Dropout(0.2)(holder)
outputs = prediction_layer(holder)
pre_model = tf.keras.Model(inputs, outputs)

pre_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics=['accuracy'])

pre_gen_model = keras.models.clone_model(pre_model)
pre_gen_model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.RMSprop(learning_rate= 1e-4),
              metrics= ['accuracy'])

pre_model.summary()
pre_gen_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem_10  (None, 128, 128, 3)      0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_10 (TFOpLamb  (None, 128, 128, 3)      0         
 da)                                                             
                                                                 
 inception_v3 (Functional)   (None, 2, 2, 2048)        21802784  
                                                                 
 global_average_pooling2d_10  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                          

# Results and Discussion

The following results were achieved ...

In [ ]:
import time
EPOCHS = 4
VERBOSE = 0

t0 = time.time()

hist = model.fit(
  train_ds,
  validation_data= val_ds,
  verbose = VERBOSE,
  epochs = EPOCHS,
)

print("First model training time:",time.time() - t0) # around 30 secs per epoch

In [ ]:
t0 = time.time()

gen_hist = gen_model.fit(
  train_gen_ds,
  validation_data= val_gen_ds,
  verbose = VERBOSE,
  epochs = round(EPOCHS),
)
print("Second model training time:",time.time() - t0) # around 120 secs per epoch

In [ ]:
t0 = time.time()

pre_hist = pre_model.fit(
  train_ds,
  validation_data= val_ds,
  verbose = VERBOSE,
  epochs = round(EPOCHS),
)
print("Third model training time:",time.time() - t0) # around 140 secs per epoch

In [ ]:
t0 = time.time()

pre_gen_hist = pre_gen_model.fit(
  train_gen_ds,
  validation_data= val_gen_ds,
  verbose = VERBOSE,
  epochs = round(EPOCHS),
)
print("Fourth model training time:",time.time() - t0) # around 240 secs per epoch

# Conclusions

In [ ]:
from IPython.core.pylabtools import figsize
from matplotlib.pyplot import figure, subplot, plot, title, ylim, legend, grid

x_axis = np.arange(len(hist.history['loss'])) + 1


figure(figsize(13,5))
subplot(1,2,1)
plot(x_axis, hist.history['loss'], 'x-', label = 'training')
plot(x_axis, hist.history['val_loss'], 'o-', label = 'validation')
plot(x_axis, gen_hist.history['loss'], 'x-', label = 'gen training')
plot(x_axis, gen_hist.history['val_loss'], '+-', label = 'gen validation')
title('loss')
ylim(0,)
legend()
grid()


subplot(1,2,2)
plot(x_axis, hist.history['accuracy'], 'x-', label = 'training')
plot(x_axis, hist.history['val_accuracy'], 'o-', label = 'validation')
plot(x_axis, gen_hist.history['accuracy'], '*-', label = 'gen training')
plot(x_axis, gen_hist.history['val_accuracy'], '+-', label = 'gen validation')
title('accuracy')
ylim(0,1.0)
legend()
grid()

In [ ]:
x_axis = np.arange(len(hist.history['loss'])) + 1

figure(figsize(13,5))
subplot(1,2,1)
plot(x_axis, pre_hist.history['loss'], 'x-', label = 'pre training')
plot(x_axis, pre_hist.history['val_loss'], 'o-', label = 'pre validation')
plot(x_axis, pre_gen_hist.history['loss'], 'x-', label = 'pre gen training')
plot(x_axis, pre_gen_hist.history['val_loss'], '+-', label = 'pre gen validation')
title('loss')
ylim(0,)
legend()
grid()


subplot(1,2,2)
plot(x_axis, pre_hist.history['accuracy'], 'x-', label = 'pre raining')
plot(x_axis, pre_hist.history['val_accuracy'], 'o-', label = 'pre validation')
plot(x_axis, pre_gen_hist.history['accuracy'], '*-', label = 'pre gen training')
plot(x_axis, pre_gen_hist.history['val_accuracy'], '+-', label = 'pre gen validation')
title('accuracy')
ylim(0,1.0)
legend()
grid()

In [ ]:
print("Train Loss = ", 100 * (hist.history['val_loss'][EPOCHS - 1]), "%")
print("Train Accuracy = ", 100 * (hist.history['val_accuracy'][EPOCHS -1]),"%")

print("\nGenerated data Train Loss = ", 100 * (gen_hist.history['val_loss'][EPOCHS -1]), "%")
print("Generated data Train Accuracy = ", 100 * (gen_hist.history['val_accuracy'][EPOCHS -1]),"%")

print("\nPretrained model Train Loss = ", 100 * (pre_hist.history['val_loss'][EPOCHS - 1]), "%")
print("Pretrained model Train Accuracy = ", 100 * (pre_hist.history['val_accuracy'][EPOCHS -1]),"%")

print("\nGenerated data Train Loss = ", 100 * (gen_hist.history['val_loss'][EPOCHS -1]), "%")
print("Generated data Train Accuracy = ", 100 * (gen_hist.history['val_accuracy'][EPOCHS -1]),"%")

In [ ]:
results = model.evaluate(test_ds,verbose=VERBOSE)
print("Model 1 test acc:", 100 * results[1])

results = gen_model.evaluate(test_ds,verbose=VERBOSE)
print("Model 2 test acc:", 100 * results[1])

results = pre_model.evaluate(test_ds,verbose=VERBOSE)
print("Model 3 test acc:", 100 * results[1])

results = pre_gen_model.evaluate(test_ds,verbose=VERBOSE)
print("Model 4 test acc:", 100 * results[1])

We used transfer method for this model. We tried to find pretrained model that worked best with our usecase. With first a google search what are the best pretrained models for our case we got a list of ones that are commonly used. We started testing the ones on that list to find out what is the best model for us. With all we used Imagenet as weights and had a very simple and few layer child model. First we tried some from the resnet family. ResNet50 and ResNet101 gave us similiar results but ResNet50 was alot faster. After that we tried VGG16 and VGG19. Those models werent too good for our case and gave us worst results compared to ResNets and also took little longer. They also require more memory  and are alot bigger sized models. Lastly we tried Inception family models. InceptionV3 and InceptionResNetV2 were our choises.